# Generating UI (data visualizations) for index.html

In [1]:
import sqlite3
import pandas as pd
import json


# connect to database

conn = sqlite3.connect("Submission.db")

Idea: Create an interactive bar graph matrix

Based on the attributes, we create a bar graph matrix comparing injury vs non injury based on weather

Each bar graph is then clickable, which opens a secondary stratified line graph (by court type)

In [3]:
def generic_bar(): # generic bar graph (injuries per court type - 3 layers)
    # Filter only those results from July (07) 2022 to February (02) 2023
    query = "SELECT Surface, Injury FROM Data WHERE Date BETWEEN '2022-07-01' AND '2023-02-28'"

    df = pd.read_sql_query(query, conn)

    # counting injuries per surface type
    grouped_data = df.groupby(['Surface', 'Injury']).size().reset_index(name='Count')

    # extract all the counts for the chart
    count_injured_hard = grouped_data[(grouped_data['Surface'] == 'Hard') & (grouped_data['Injury'] == 1)]['Count'].values[0] 
    count_non_injured_hard = grouped_data[(grouped_data['Surface'] == 'Hard') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    count_injured_clay = grouped_data[(grouped_data['Surface'] == 'Clay') & (grouped_data['Injury'] == 1)]['Count'].values[0]
    count_non_injured_clay = grouped_data[(grouped_data['Surface'] == 'Clay') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    count_injured_grass = grouped_data[(grouped_data['Surface'] == 'Grass') & (grouped_data['Injury'] == 1)]['Count'].values[0]
    count_non_injured_grass = grouped_data[(grouped_data['Surface'] == 'Grass') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    # dump to JSON
    # wrapper for each variable in order to make it JSON serializable

    data = {
        "Hard": {
            "Injured": int(count_injured_hard),
            "Non-Injured": int(count_non_injured_hard)
        },
        "Clay": {
            "Injured": int(count_injured_clay),
            "Non-Injured": int(count_non_injured_clay)
        },
        "Grass": {
            "Injured": int(count_injured_grass),
            "Non-Injured": int(count_non_injured_grass)
        }
    }

    with open("injury_vs_court_type.json", "w") as f:
        json.dump(data, f)
        
generic_bar()        

In [ ]:
# close connection
conn.close()